In [1]:
from glob import glob
from tqdm import tqdm
import os, json
from pathlib import Path
from shutil import copy2

In [3]:
base_path = '/home/brad/Dataset/persona-montage/data/1.Training/라벨링데이터/TL/'

json_list = glob(os.path.join(base_path, 'json/*/*.json'))

print(len(json_list))

24213


In [4]:
for json_path in json_list:
    with open(json_path, 'r') as f:
        meta_data = json.load(f)

    print(meta_data)
    break

{'info': [{'org_id': 4197, 'org_img_name': '4197', 'org_path': '/Images/montage/4197.png', 'org_img_size': 153, 'org_img_pix': '540x720', 'age': 55, 'gender': 'M', 'level': 3}], 'descripton': [{'org_id': 4197, 'face': {'type': '긴형', 'size': '크다', 'foreheadType': '알수없음', 'foreheadSize': '넓다', 'chinType': '주걱형', 'chinSize': '길다', 'cheek': '광대돌출(예: 비대칭은 분류 없음 선택 후 기술)', 'description': '앞머리가 왼쪽 이마를 살작 가려 이마유형은 알수 없으나 오른쪽 이마 크기를 볼때 넓다는걸 짐작 할수 있다.  앞 광대가 돌출되어 있어 빛이 반사되어 있다. '}, 'hairstyle': {'type': '곱슬머리', 'topLength': '앞머리올림', 'sizeLength': '숏컷(귀보임)', 'part': '오른쪽가르마', 'description': '2대 8 오른쪽 가르마로 풍성하게 드라이 한 듯한 모발로 옆으로 넘긴 머리 모양이다.  '}, 'eyebrows': {'type': '각진형', 'deep': '흐리다', 'length': '보통', 'thick': '보통', 'glabella': '보통', 'description': '눈썹은 각진형이며 오른쪽 눈썹은 3분에 2지점으로 갈수록 얇아지는 눈썹 형태고 왼쪽 눈썹은 잔털이 많이 있다.'}, 'eyes': {'size': '보통', 'type': '외꺼풀', 'distance': '보통', 'slant': '내려감', 'shape': '오목눈', 'eyelids': '보통', 'bottom': '있다', 'description': '눈두덩이에 살이 많고 눈썹 바로 아래가 움푹 들어간 눈 모양이다. 눈꼬리가 내려가 있으나

In [6]:
for json_path in json_list:
    with open(json_path, 'r') as f:
        meta_data = json.load(f)

    print(meta_data['descripton'][0]['impression']['description'])
    print(meta_data['sketch_info'][0]['img_path'])
    print(meta_data['org_sketch_info'][0]['img_path'])
    break

단정하게 넘긴 풍성한 헤어 스타일과 깨끗한 피부는 깔끔한 인상을 주며 미소를 먹은 듯한 입술에선 친절한 미소가 보인다. 쌍꺼풀이 없는 눈매이긴 하나 눈이 오목하게 동글하게 생겨 부드럽게 보인다.
/Images/sketch/H/4197_12002.jpg
/Images/org_sketch/4197.png


In [17]:
target_path = '/home/brad/Dataset/persona-montage/preprocessed'

img_list = []
label_list = []

os.makedirs(target_path, exist_ok=True)
os.makedirs(os.path.join(target_path, 'images'), exist_ok=True)
os.makedirs(os.path.join(target_path, 'labels'), exist_ok=True)

for i, json_path in enumerate(tqdm(json_list)):
    with open(json_path, 'r') as f:
        meta_data = json.load(f)

    if 'impression' in meta_data['descripton'][0]:
        text = meta_data['descripton'][0]['impression']['description']
    else:
        # print(meta_data['descripton'][0])
        text = meta_data['descripton'][0]['face']['description']

    img_path = os.path.join(base_path, meta_data['sketch_info'][0]['img_path'][1:])
    orig_img_path = os.path.join(base_path, meta_data['org_sketch_info'][0]['img_path'][1:])

    target_img_path = os.path.join(target_path, 'images', str(i).zfill(5) + '_' + os.path.splitext(os.path.basename(json_path))[0] + '.jpg')
    target_orig_img_path = os.path.join(target_path, 'images', str(i).zfill(5) + '_' + 'orig_' + os.path.splitext(os.path.basename(json_path))[0] + '.png')

    target_label_path = os.path.join(target_path, 'labels', str(i).zfill(5) + '_' + os.path.splitext(os.path.basename(json_path))[0] + '.txt')
    target_orig_label_path = os.path.join(target_path, 'labels', str(i).zfill(5) + '_' + 'orig_' + os.path.splitext(os.path.basename(json_path))[0] + '.txt')

    img_list.append(target_img_path)
    img_list.append(target_orig_img_path)
    label_list.append(target_label_path)
    label_list.append(target_orig_label_path)

    copy2(img_path, target_img_path)
    copy2(orig_img_path, target_orig_img_path)

    with open(target_label_path, 'w') as f:
        f.write(text)
    with open(target_orig_label_path, 'w') as f:
        f.write(text)

print(len(img_list), len(label_list))

100%|██████████| 24213/24213 [00:07<00:00, 3389.45it/s]

48426 48426


In [18]:
import numpy as np

np.savetxt(os.path.join(target_path, 'image_list.txt'), img_list, fmt='%s')
np.savetxt(os.path.join(target_path, 'label_list.txt'), label_list, fmt='%s')

In [22]:
image_path = Path(os.path.join(target_path))
image_files = [
    *image_path.glob("**/*[0-9].txt"),
    *image_path.glob("**/*[0-9].jpg"),
    *image_path.glob("**/*[0-9].png"),
]

len(image_files) / 2

48426.0

In [7]:
val_img_list = glob(os.path.join('/home/brad/Dataset/persona-montage/preprocessed', 'val/*.jpg'))

len(val_img_list)

1009

In [8]:
import numpy as np

np.savetxt(os.path.join('/home/brad/Dataset/persona-montage/preprocessed', 'image_list_test.txt'), val_img_list, fmt='%s')